# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [1]:
%%file count_letters.py
from collections import Counter

def count_letters(file):
    
    with open(file,encoding="windows-1251") as fp:
        text = fp.read().lower()
        return Counter(text)

Overwriting count_letters.py


In [2]:
from count_letters import count_letters

In [3]:
%%time
count_letters("./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt")
count_letters("./data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt")

CPU times: user 98.5 ms, sys: 4.11 ms, total: 103 ms
Wall time: 103 ms


Counter({'с': 50084,
         'п': 25652,
         'а': 73555,
         'и': 62030,
         'б': 16016,
         'о': 106740,
         ',': 26973,
         ' ': 182305,
         'ч': 16492,
         'т': 59813,
         'к': 30802,
         'л': 42328,
         'н': 60920,
         'г': 16174,
         'у': 27309,
         'в': 43700,
         'е': 80972,
         'й': 9747,
         'э': 3203,
         'р': 39784,
         'b': 25,
         'o': 104,
         'k': 16,
         's': 96,
         'c': 42,
         'a': 98,
         'f': 23,
         'e': 162,
         '.': 9864,
         'n': 114,
         't': 98,
         ':': 984,
         'h': 48,
         'p': 29,
         '/': 22,
         '\n': 8583,
         'u': 86,
         'r': 76,
         'd': 38,
         'v': 65,
         'i': 235,
         'y': 5,
         '_': 8,
         '-': 3558,
         '1': 384,
         '0': 110,
         '9': 100,
         '6': 271,
         'm': 54,
         'l': 46,
         'ж': 10552,
     

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [4]:
import multiprocessing as mp

In [5]:
if __name__ == "__main__":
    files = ["./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt", "./data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt"]
    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files)
    

In [6]:
len(counters)

2

In [7]:
counters

[Counter({'с': 11507,
          'п': 5489,
          'а': 18236,
          'и': 13587,
          'б': 3980,
          'о': 23130,
          ',': 6372,
          ' ': 45076,
          'ч': 4113,
          'т': 14245,
          'к': 6744,
          'л': 9961,
          'н': 14240,
          'г': 3948,
          'у': 6044,
          'в': 9398,
          'е': 20054,
          'й': 2028,
          'э': 836,
          'р': 9482,
          'b': 220,
          'o': 377,
          'k': 21,
          's': 429,
          'c': 324,
          'a': 590,
          'f': 52,
          'e': 1200,
          '.': 2954,
          'n': 459,
          't': 332,
          ':': 212,
          'h': 227,
          'p': 100,
          '/': 20,
          '\n': 2734,
          'u': 285,
          'r': 308,
          'd': 192,
          'v': 87,
          'i': 369,
          'y': 8,
          '_': 4,
          '-': 900,
          '1': 46,
          '0': 22,
          '9': 36,
          '6': 42,
          'm': 401,
 

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [8]:
import csv
import math

In [9]:
files_count = 8
with open('./data/recipes_full.csv') as file:
    reader = csv.reader(file)
    all_rows_count = sum(1 for row in reader)-1
single_rows_count = int(math.ceil(all_rows_count/files_count))
print(all_rows_count, single_rows_count)

2231637 278955


In [10]:
points_list = [all_rows_count]
left = all_rows_count

files_ranges = {}
for i in range(files_count):
    left = left - single_rows_count
    points_list.append(left)

points_list[-1] = 0
points_list = points_list[::-1]
points_list

[0, 278952, 557907, 836862, 1115817, 1394772, 1673727, 1952682, 2231637]

In [11]:
files_path_dict = {i: {"name": f"./out/id_tag_nsteps_{i}.csv"} for i in range(1,files_count+1)}
for i in range(0, len(points_list)-1):
    x, y = points_list[i], points_list[i+1]
    files_path_dict[i+1]["min_point"] = x
    files_path_dict[i+1]["max_point"] = y

files_path_dict

{1: {'name': './out/id_tag_nsteps_1.csv', 'min_point': 0, 'max_point': 278952},
 2: {'name': './out/id_tag_nsteps_2.csv',
  'min_point': 278952,
  'max_point': 557907},
 3: {'name': './out/id_tag_nsteps_3.csv',
  'min_point': 557907,
  'max_point': 836862},
 4: {'name': './out/id_tag_nsteps_4.csv',
  'min_point': 836862,
  'max_point': 1115817},
 5: {'name': './out/id_tag_nsteps_5.csv',
  'min_point': 1115817,
  'max_point': 1394772},
 6: {'name': './out/id_tag_nsteps_6.csv',
  'min_point': 1394772,
  'max_point': 1673727},
 7: {'name': './out/id_tag_nsteps_7.csv',
  'min_point': 1673727,
  'max_point': 1952682},
 8: {'name': './out/id_tag_nsteps_8.csv',
  'min_point': 1952682,
  'max_point': 2231637}}

In [30]:
current_iter = 0 
current_file_counter = 1

current_file_path = files_path_dict[current_file_counter]["name"]
max_point = files_path_dict[current_file_counter]["max_point"]
current_file = open(current_file_path, "w")
current_file.write("id;tag;n_steps\n")

with open('./data/recipes_full.csv') as file:
    reader = csv.reader(file)
    header = next(reader)
    
    for row in reader:
        
        name, r_id ,minutes, contributor_id, submitted, tags, n_steps, steps, description, ingredients, n_ingredients = row
        tags = eval(tags)
        #ast literal eval
        
        for tag in tags:
            current_file.write(f"{r_id};{tag};{n_steps}\n")
        
        
        current_iter += 1

        #Если в текущем файле забили все строки - переключаемся на след файл
        if current_iter == max_point:
            
            current_file.close()
            
            print(current_iter)
            print(current_file_counter)
            
            current_file_counter += 1
            
            if current_file_counter in files_path_dict:
                print("current_file_counter",current_file_counter)
                current_file_path = files_path_dict[current_file_counter]["name"]
                max_point = files_path_dict[current_file_counter]["max_point"]
                current_file = open(current_file_path, "w")
                current_file.write("id;tag;n_steps\n")
        

278952
1
current_file_counter 2
557907
2
current_file_counter 3
836862
3
current_file_counter 4
1115817
4
current_file_counter 5
1394772
5
current_file_counter 6
1673727
6
current_file_counter 7
1952682
7
current_file_counter 8
2231637
8


In [13]:
counter = 0
with open('./data/recipes_full.csv') as file:
    reader = csv.reader(file)
    for row in reader:
        counter += 1

print(counter)

2231638


In [14]:
for check_file in files_path_dict.values():
    with open(check_file["name"]) as file:
        result = sum(1 for row in file)
        print(f"{check_file['name']} -> {result}")

./out/id_tag_nsteps_1.csv -> 1771656
./out/id_tag_nsteps_2.csv -> 1769258
./out/id_tag_nsteps_3.csv -> 1764642
./out/id_tag_nsteps_4.csv -> 1767974
./out/id_tag_nsteps_5.csv -> 1763976
./out/id_tag_nsteps_6.csv -> 1765457
./out/id_tag_nsteps_7.csv -> 1769544
./out/id_tag_nsteps_8.csv -> 1766618


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [15]:
from typing import Dict

In [16]:
def my_counter(file_name: str) -> Dict[str, float]:
    results_dict = {}
    
    with open(file_name) as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            r_id, tag, n_steps = row[0].split(";")
            
            if tag not in results_dict:
                results_dict[tag]= {"count": 0,"sum" : 0}
            results_dict[tag]["count"] += 1
            results_dict[tag]["sum"] += int(n_steps)

    for tag, data in results_dict.items():
        results_dict[tag] = data["sum"]/data["count"]

    return results_dict

In [17]:
my_counter("./out/id_tag_nsteps_1.csv")

{'mexican': 5.302503052503052,
 'healthy-2': 6.328114004222378,
 'orange-roughy': 3.4451697127937337,
 'chicken-thighs-legs': 4.184877440797673,
 'freezer': 3.9144921718185466,
 'whitefish': 3.5132206328565236,
 'pork-sausage': 4.285714285714286,
 'brunch': 6.900977198697069,
 'ham-and-bean-soup': 3.5126970227670755,
 'colombian': 3.5766456266907123,
 'savory-pies': 4.294961081523965,
 'refrigerator': 4.74200503054258,
 'australian': 4.25384024577573,
 'served-cold': 4.940726577437858,
 'spaghetti': 4.1286239281339325,
 'passover': 3.6305130513051305,
 'quick-breads': 4.969485903814262,
 'californian': 3.7421907538525616,
 'namibian': 3.495145631067961,
 'candy': 4.302595893064703,
 'independence-day': 4.143187066974596,
 'baking': 3.5864793678665494,
 'pennsylvania-dutch': 3.536219459922846,
 'weeknight': 7.406489318823056,
 '60-minutes-or-less': 9.344522810463351,
 'time-to-make': 9.248938170735661,
 'course': 9.242875343659895,
 'cuisine': 9.133115942028985,
 'preparation': 9.293666

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [18]:
from functools import reduce
import time

In [31]:
files_list = [item["name"] for item in files_path_dict.values()]
files_list

['./out/id_tag_nsteps_1.csv',
 './out/id_tag_nsteps_2.csv',
 './out/id_tag_nsteps_3.csv',
 './out/id_tag_nsteps_4.csv',
 './out/id_tag_nsteps_5.csv',
 './out/id_tag_nsteps_6.csv',
 './out/id_tag_nsteps_7.csv',
 './out/id_tag_nsteps_8.csv']

In [20]:
%%file my_counter_new.py
from typing import Dict
import csv
def my_counter_new(file_name: str) -> Dict[str, Dict[str, int]]:
    """Считает кол-во и сумму шагов в тегах переданного файла"""
    results_dict = {}
    with open(file_name) as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            r_id, tag, n_steps = row[0].split(";")
            
            if tag not in results_dict:
                results_dict[tag]= {"count": 0,"sum" : 0}
            results_dict[tag]["count"] += 1
            results_dict[tag]["sum"] += int(n_steps)

    return results_dict

Overwriting my_counter_new.py


In [21]:
from my_counter_new import my_counter_new

In [22]:
def merger(d1 : Dict[str, int], d2 : Dict[str, int]) -> Dict[str, int]:
    """Метод для объединения словарей"""
    for tag, data in d1.items():
        if tag in d2:
            d2[tag]["sum"] += data["sum"]
            d2[tag]["count"] += data["count"]
        else:
            d2[tag] = data
    
    return d2

In [23]:
def mean_counter(ddict: Dict[str, int]) -> Dict[str, float]:
    """Метод для подсчета среднего значения"""
    for tag, data in ddict.items():
        ddict[tag] = data["sum"]/data["count"]
    return ddict

In [24]:
start_time = time.time()
result_dict = reduce(lambda x, y: merger(x,y), [my_counter_new(item) for item in files_list])
print(mean_counter(result_dict)["lunch"])
print(f"--- {time.time() - start_time} сек ---")

6.678860028860029
--- 8.420361995697021 сек ---


4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [32]:
if __name__ == "__main__":
    start_time = time.time()
    with mp.Pool(processes=len(files_list)) as pool:
        counters = pool.map(my_counter_new, files_list)

    result_dict = reduce(lambda x, y: merger(x,y), counters)
    print(mean_counter(result_dict)["lunch"])
    print(f"--- {time.time() - start_time} сек ---")

6.678860028860029
--- 1.823716163635254 сек ---


5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [26]:
import multiprocessing as mp

In [27]:
%%file my_worker.py
from typing import Dict
import csv
from multiprocessing import Process

class MyWorker(Process):
    """Обработчик тасков"""

    def __init__(self, input_q, result_q):
        Process.__init__(self)
        self.input_q = input_q
        self.result_q = result_q

    def run(self):
        while not self.input_q.empty():
            current_file = self.input_q.get()
            processing_result = self.my_counter_new(current_file)
            self.result_q.put(processing_result)

    def my_counter_new(self, file_name: str) -> Dict[str, Dict[str, int]]:
        """Считает кол-во и сумму шагов в тегах переданного файла"""
        results_dict = {}
        with open(file_name) as file:
            reader = csv.reader(file)
            header = next(reader)
            for row in reader:
                r_id, tag, n_steps = row[0].split(";")

                if tag not in results_dict:
                    results_dict[tag]= {"count": 0,"sum" : 0}
                results_dict[tag]["count"] += 1
                results_dict[tag]["sum"] += int(n_steps)
        
        return results_dict

Overwriting my_worker.py


In [28]:
from my_worker import MyWorker

In [29]:
if __name__ == "__main__":
    
    start_time = time.time()
    P_COUNT = 4
    processes_list = []
    
    task_q = mp.Queue()
    result_q = mp.Queue()
    
    for file in files_list:
        task_q.put(file)
        
    for i in range(P_COUNT):
        p = MyWorker(task_q,result_q)
        p.start()
        processes_list.append(p)
    
        
    # Очередь ответов
    results_list = [result_q.get() for _ in range(P_COUNT*2)]
    result_dict = reduce(lambda x, y: merger(x,y), results_list)
    print(mean_counter(result_dict)["lunch"])
    print(f"--- {time.time() - start_time} сек ---")

6.678860028860029
--- 2.2447922229766846 сек ---
